# Validation of gens_eia860
This notebook runs sanity checks on the Generators data that are reported in EIA Form 860. These are the same tests which are run by the gens_eia860 validation tests by PyTest.  The notebook and visualizations are meant to be used as a diagnostic tool, to help understand what's wrong when the PyTest based data validations fail for some reason.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import pandas as pd
import sqlalchemy as sa
import pudl

In [ ]:
import warnings
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

In [ ]:
plt.style.use('ggplot')
mpl.rcParams['figure.figsize'] = (10,4)
mpl.rcParams['figure.dpi'] = 150
pd.options.display.max_columns = 56

In [ ]:
pudl_settings = pudl.workspace.setup.get_defaults()
ferc1_engine = sa.create_engine(pudl_settings['ferc1_db'])
pudl_engine = sa.create_engine(pudl_settings['pudl_db'])

## Get the original EIA 860 data
First we pull the original (post-ETL) EIA 860 data out of the database. We will use the values in this dataset as a baseline for checking that latter aggregated data and derived values remain valid.  We will also eyeball these values here to make sure they are within the expected range. This may take a minute or two depending on the speed of your machine.

In [ ]:
pudl_out_orig = pudl.output.pudltabl.PudlTabl(pudl_engine, freq=None)
gens_eia860_orig = pudl_out_orig.gens_eia860()

# Validation Against Fixed Bounds 
Some of the variables reported in this table have a fixed range of reasonable values, like the capacity.  These varaibles can be tested for validity against external standards directly.  In general we have two kinds of tests in this section:
* **Tails:** are the exteme values too extreme? Typically, this is at the 5% and 95% level, but depending on the distribution, sometimes other thresholds are used.
* **Middle:** Is the central value of the distribution where it should be?

### Fields that need checking:
The main column we are checking right now is the capacity.

Future data-like columns to potentially check:
  - 'minimum_load_mw'
  - 'nameplate_power_factor'
  - 'planned_net_summer_capacity_derate_mw'
  - 'planned_net_summer_capacity_uprate_mw'
  - 'planned_net_winter_capacity_derate_mw'
  - 'planned_net_winter_capacity_uprate_mw'
  - 'planned_new_capacity_mw'
  - 'summer_capacity_mw'
  - 'winter_capacity_mw'

In [ ]:
gens_eia860_orig.sample(10)

## Capacity

In [ ]:
pudl.validate.plot_vs_bounds(gens_eia860_orig, pudl.validate.gens_eia860_vs_bound)

# Validating Historical Distributions
As a sanity check of the testing process itself, we can check to see whether the entire historical distribution has attributes that place it within the extremes of a historical subsampling of the distribution. In this case, we sample each historical year, and look at the range of values taken on by some quantile, and see whether the same quantile for the whole of the dataset fits within that range

In [ ]:
pudl.validate.plot_vs_self(gens_eia860_orig, pudl.validate.gens_eia860_self)